In [6]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten
from keras.models import Model
from keras.layers import Input
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D

from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

from keras.layers.wrappers import TimeDistributed

import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf
import glob

import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd
import random

In [79]:

def getModel2( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

#     for layer in vgg_model.layers:
#         layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model    

def load_model(weights_path, output_dim):

    model = getModel2( output_dim ) 
    for k,layer in model.layers_by_depth.items()[:]:
        layer[0].trainable = False
#     for layer in model.layers:
#         layer.trainable = False
        
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model

def load_rnn_model(weights):
    model_rnn3=Sequential()

    model_rnn3.add(TimeDistributed(trained_model, input_shape=(max_frames,224,224,3)))
    model_rnn3.add(GRU(output_dim=100,return_sequences=True))
    model_rnn3.add(GRU(output_dim=50,return_sequences=False))
    model_rnn3.add(Dropout(.2))
    model_rnn3.add(Dense(number_of_brands,activation='softmax'))
    model_rnn3.load_weights(weights)

    model_rnn3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy', 'categorical_crossentropy'])

    return model_rnn3

labels_dict = {'apple': 2, 'cocacola': 0, 'nike': 4, 'pepsi': 1, 'starbucks': 3}
reversed_dict = dict([(x[1],x[0]) for x in labels_dict.items()])

def get_brand(softmaxes):
    v = np.max(softmaxes.mean(axis=0))
    if v > 0.6:
        return reversed_dict[np.argmax(softmaxes.mean(axis=0))]
    return "Noise"


In [8]:
trained_model = load_model("weights-improvement-VGG16-LOGO5-04-0.9940.hdf5", 5)

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [11]:
max_frames = 5
number_of_brands = 5
rnn_model = load_rnn_model("RNN-05-0.8704.hdf5")

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:47: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=100, return_sequences=True)`
/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:48: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=50, return_sequences=False)`


In [12]:
## Download Video

In [131]:
url = "https://www.youtube.com/watch?v=vWD2U0CUB8o"

In [132]:
from pytube import YouTube
yt = YouTube(url)

In [133]:
video = yt.get('mp4', '720p')

In [134]:
cmd1 = "rm /tmp/tmp_1"
sp.call(cmd1,shell=True)
video.download('/tmp/tmp_1')

In [135]:
## Format video

In [136]:
import subprocess as sp
import sys

In [137]:
filename = "/tmp/tmp_1"

cmd1 = "rm -r /tmp/tmp_1_1"
sp.call(cmd1,shell=True)

cmd1 = "mkdir /tmp/tmp_1_1"
sp.call(cmd1,shell=True)
outputpath= "/tmp/tmp_1_1/"
        
outputfile = outputpath + "f"
cmd2='ffmpeg -i '+filename+' -vframes 100  -r 2 -s 224x224 ' + outputfile + '_%04d.jpg'
sp.call(cmd2,shell=True)
sys.stdout.write("Saved it at " + outputfile +"\n")

Saved it at /tmp/tmp_1_1/f


In [138]:
## Get top sliding window

In [139]:
import glob

In [140]:
frames = glob.glob("/tmp/tmp_1_1/*")
x_mean = 0.39533365588770686

In [141]:
imgs = []
for frame in frames:
    img = np.asarray(Image.open(frame))/255.
    imgs.append(img)
    
imgs = np.array(imgs) - x_mean
    
    
    
    

In [142]:
softmaxes = trained_model.predict(imgs)

In [145]:
softmaxes

array([[  1.13027580e-02,   9.85687554e-01,   3.41692619e-04,
          2.89327204e-06,   2.66503519e-03],
       [  1.59143692e-05,   2.26134494e-01,   7.60171175e-01,
          9.51263337e-06,   1.36689423e-02],
       [  2.86663471e-10,   2.40629817e-07,   9.99999404e-01,
          1.74327173e-13,   3.20528869e-07],
       [  2.45911215e-04,   7.71836579e-01,   1.62884146e-01,
          2.30583100e-05,   6.50102645e-02],
       [  5.65120445e-07,   1.16335330e-04,   9.99877572e-01,
          3.18504357e-15,   5.52421534e-06],
       [  6.17952883e-01,   3.81364912e-01,   6.26906456e-07,
          2.91565811e-07,   6.81250822e-04],
       [  2.29232285e-07,   8.31683341e-04,   9.02126670e-01,
          8.62154259e-09,   9.70413685e-02],
       [  2.53962865e-03,   9.95535970e-01,   9.58174351e-04,
          7.15285807e-07,   9.65528248e-04],
       [  5.31988358e-03,   3.98098491e-02,   5.04799536e-04,
          6.04517911e-12,   9.54365432e-01],
       [  7.29533611e-03,   9.8336267

In [144]:
avgs = []
for i in range(0, len(softmaxes)-max_frames ):
    avgs.append(softmaxes[i:i+max_frames].mean(axis=0))
    
avgs = np.array(avgs)
top_idxs = []
for i in range(number_of_brands):
    #print np.argmax(avgs[:,i])
    top_idxs.append(np.argmax(avgs[:,i]))



In [146]:
sequences = []

for top_f in top_idxs:
    sequences.append(imgs[top_f:top_f+max_frames])

sequences = np.array(sequences)

In [147]:
sequences.shape

(5, 5, 224, 224, 3)

In [148]:
ps = rnn_model.predict(sequences)

In [151]:
ps

array([[ 0.10456314,  0.19670339,  0.2299431 ,  0.01223137,  0.45655906],
       [ 0.05979851,  0.79685622,  0.04873443,  0.02369591,  0.07091497],
       [ 0.05335395,  0.46952891,  0.4394865 ,  0.00750573,  0.03012501],
       [ 0.04925825,  0.14136173,  0.11072086,  0.01747943,  0.68117976],
       [ 0.00798998,  0.03510812,  0.0463621 ,  0.00384198,  0.90669787]], dtype=float32)

In [155]:
np.argmax(ps)/ number_of_brands

4

In [150]:
ps.flatten()[np.argmax(ps)]

0.90669787

In [108]:
avgs[:,1]

array([  6.05653549e-05,   1.38390504e-04,   1.38393138e-04,
         1.39819138e-04,   1.40504621e-04,   2.21002410e-04,
         6.70960348e-04,   8.83733388e-04,   3.39805149e-03,
         1.89154729e-01,   1.89013779e-01,   1.88486010e-01,
         1.89327389e-01,   1.86811626e-01,   1.05423713e-03,
         4.82060295e-03,   4.82058711e-03,   3.80246597e-03,
         4.56987787e-03,   6.46532401e-02,   6.08869903e-02,
         6.08869977e-02,   6.08509853e-02,   6.02137931e-02,
         1.30432265e-04,   1.20727420e-01,   1.20729826e-01,
         1.20738350e-01,   1.26925290e-01,   1.26925886e-01,
         6.32875692e-03,   6.32633548e-03,   6.31779805e-03,
         1.35171562e-04,   1.34621660e-04,   1.44198682e-04,
         1.44197227e-04,   1.44580816e-04,   1.06869929e-05,
         1.06319567e-05,   6.50129659e-05,   2.23479699e-04,
         2.23234150e-04,   2.22587769e-04,   2.22577102e-04,
         1.58618699e-04,   4.76569539e-06,   4.69076667e-06,
         3.37898091e-04,

In [93]:
get_brand(softmaxes)

'Noise'

In [62]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten
from keras.models import Model
from keras.layers import Input
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D

from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

from keras.layers.wrappers import TimeDistributed

import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf
import glob

import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd
import random
import subprocess as sp
import sys
import glob
from pytube import YouTube
import pytube
max_frames = 5
number_of_brands = 5


def getModel2( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

#     for layer in vgg_model.layers:
#         layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model    

def load_model(weights_path, output_dim):

    model = getModel2( output_dim ) 
    for k,layer in model.layers_by_depth.items()[:]:
        layer[0].trainable = False
#     for layer in model.layers:
#         layer.trainable = False
        
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model


labels_dict = {'apple': 2, 'cocacola': 0, 'nike': 4, 'pepsi': 1, 'starbucks': 3}
reversed_dict = dict([(x[1],x[0]) for x in labels_dict.items()])

def get_brand(softmaxes):
    v = np.max(softmaxes.mean(axis=0))
    
    print v
    
    print reversed_dict[np.argmax(softmaxes.mean(axis=0))]
    if v > 0.6:
        return reversed_dict[np.argmax(softmaxes.mean(axis=0))]
    return "Noise"


def get_brand_2(results):
    v = np.argmax(results)/ number_of_brands
    print np.max(results)
    return reversed_dict[v]


def load_rnn_model(weights, cnn):
    model_rnn3=Sequential()

    model_rnn3.add(TimeDistributed(cnn, input_shape=(max_frames,224,224,3)))
    model_rnn3.add(GRU(output_dim=100,return_sequences=True))
    model_rnn3.add(GRU(output_dim=50,return_sequences=False))
    model_rnn3.add(Dropout(.2))
    model_rnn3.add(Dense(number_of_brands,activation='softmax'))
    model_rnn3.load_weights(weights)

    model_rnn3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy', 'categorical_crossentropy'])

    return model_rnn3

class DeepLogo():
    def __init__(self):
        self.trained_model = load_model("weights-improvement-VGG16-LOGO5-04-0.9940.hdf5", 5)
        self.rnn_model = load_rnn_model("RNN-05-0.8704.hdf5", self.trained_model)
        

    def predict(self, url):
        yt = YouTube(url)
        try:
            video = yt.get('mp4')
        except pytube.exceptions.MultipleObjectsReturned:
            video = yt.get('mp4', '720p')
        
        
        cmd1 = "rm tmp/tmp_1"
        sp.call(cmd1,shell=True)
        video.download('tmp/tmp_1')
        
        filename = "tmp/tmp_1"

        cmd1 = "rm -r tmp/tmp_1_1"
        sp.call(cmd1,shell=True)

        cmd1 = "mkdir tmp/tmp_1_1"
        sp.call(cmd1,shell=True)
        outputpath= "tmp/tmp_1_1/"

        outputfile = outputpath + "f"
        cmd2='ffmpeg -i '+filename+' -r 1 -s 224x224 ' + outputfile + '_%04d.jpg'
        sp.call(cmd2,shell=True)
        sys.stdout.write("Saved it at " + outputfile +"\n")
        
        frames = sorted(glob.glob("tmp/tmp_1_1/*"))
        print frames
        x_mean = 0.39533365588770686
        
        imgs = []
        for frame in frames:
            img = np.asarray(Image.open(frame))/255.
            imgs.append(img)

        imgs = np.array(imgs)
        imgs = imgs - imgs.mean()

        print "predicting CNN..."
        softmaxes = self.trained_model.predict(imgs)
        b1 =  get_brand(softmaxes)
        print b1
        avgs = []
        for i in range(0, len(softmaxes)-max_frames ):
            avgs.append(softmaxes[i:i+max_frames].mean(axis=0))

        avgs = np.array(avgs)
        top_idxs = []
        for i in range(number_of_brands):
            #print np.argmax(avgs[:,i])
            top_idxs.append(np.argmax(avgs[:,i]))

        print top_idxs
        sequences = []

        for top_f in top_idxs:
            sequences.append(imgs[top_f:top_f+max_frames])

        sequences = np.array(sequences)
        print "predicting RNN..."
        ps = self.rnn_model.predict(sequences)
        b2 = get_brand_2(ps)
        if b1 == b2:
            brand =  b1
        else:
            brand = "Noise"
        return softmaxes,imgs,frames,brand,b1,b2,ps


In [63]:
dp = DeepLogo()

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:62: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:108: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=100, return_sequences=True)`
/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:109: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=50, return_sequences=False)`


In [64]:
softmaxes,imgs,frames,brand,b1,b2,ps = dp.predict("https://www.youtube.com/watch?v=QFOKEKeOhHI")

Saved it at tmp/tmp_1_1/f
['tmp/tmp_1_1/f_0001.jpg', 'tmp/tmp_1_1/f_0002.jpg', 'tmp/tmp_1_1/f_0003.jpg', 'tmp/tmp_1_1/f_0004.jpg', 'tmp/tmp_1_1/f_0005.jpg', 'tmp/tmp_1_1/f_0006.jpg', 'tmp/tmp_1_1/f_0007.jpg', 'tmp/tmp_1_1/f_0008.jpg', 'tmp/tmp_1_1/f_0009.jpg', 'tmp/tmp_1_1/f_0010.jpg', 'tmp/tmp_1_1/f_0011.jpg', 'tmp/tmp_1_1/f_0012.jpg', 'tmp/tmp_1_1/f_0013.jpg', 'tmp/tmp_1_1/f_0014.jpg', 'tmp/tmp_1_1/f_0015.jpg', 'tmp/tmp_1_1/f_0016.jpg', 'tmp/tmp_1_1/f_0017.jpg', 'tmp/tmp_1_1/f_0018.jpg', 'tmp/tmp_1_1/f_0019.jpg', 'tmp/tmp_1_1/f_0020.jpg', 'tmp/tmp_1_1/f_0021.jpg', 'tmp/tmp_1_1/f_0022.jpg', 'tmp/tmp_1_1/f_0023.jpg', 'tmp/tmp_1_1/f_0024.jpg', 'tmp/tmp_1_1/f_0025.jpg', 'tmp/tmp_1_1/f_0026.jpg', 'tmp/tmp_1_1/f_0027.jpg', 'tmp/tmp_1_1/f_0028.jpg', 'tmp/tmp_1_1/f_0029.jpg', 'tmp/tmp_1_1/f_0030.jpg', 'tmp/tmp_1_1/f_0031.jpg', 'tmp/tmp_1_1/f_0032.jpg', 'tmp/tmp_1_1/f_0033.jpg', 'tmp/tmp_1_1/f_0034.jpg', 'tmp/tmp_1_1/f_0035.jpg', 'tmp/tmp_1_1/f_0036.jpg', 'tmp/tmp_1_1/f_0037.jpg', 'tmp/tmp_1_

In [58]:
brand, b1, b2

('Noise', 'Noise', 'nike')

In [65]:
ps

array([[ 0.04669514,  0.0054139 ,  0.02663686,  0.00380472,  0.91744941],
       [ 0.02864334,  0.87409276,  0.03865713,  0.01760199,  0.04100476],
       [ 0.00741815,  0.04492484,  0.93896866,  0.00109814,  0.0075902 ],
       [ 0.01921216,  0.03456241,  0.77223134,  0.00434536,  0.16964872],
       [ 0.00194719,  0.00265841,  0.03472063,  0.00238577,  0.95828801]], dtype=float32)

In [62]:
frames = ['tmp/tmp_1_1/f_0001.jpg', 'tmp/tmp_1_1/f_0002.jpg', 'tmp/tmp_1_1/f_0003.jpg', 'tmp/tmp_1_1/f_0004.jpg', 'tmp/tmp_1_1/f_0005.jpg', 'tmp/tmp_1_1/f_0006.jpg', 'tmp/tmp_1_1/f_0007.jpg', 'tmp/tmp_1_1/f_0008.jpg', 'tmp/tmp_1_1/f_0009.jpg', 'tmp/tmp_1_1/f_0010.jpg', 'tmp/tmp_1_1/f_0011.jpg', 'tmp/tmp_1_1/f_0012.jpg', 'tmp/tmp_1_1/f_0013.jpg', 'tmp/tmp_1_1/f_0014.jpg', 'tmp/tmp_1_1/f_0015.jpg', 'tmp/tmp_1_1/f_0016.jpg', 'tmp/tmp_1_1/f_0017.jpg', 'tmp/tmp_1_1/f_0018.jpg', 'tmp/tmp_1_1/f_0019.jpg', 'tmp/tmp_1_1/f_0020.jpg', 'tmp/tmp_1_1/f_0021.jpg', 'tmp/tmp_1_1/f_0022.jpg', 'tmp/tmp_1_1/f_0023.jpg', 'tmp/tmp_1_1/f_0024.jpg', 'tmp/tmp_1_1/f_0025.jpg', 'tmp/tmp_1_1/f_0026.jpg', 'tmp/tmp_1_1/f_0027.jpg', 'tmp/tmp_1_1/f_0028.jpg', 'tmp/tmp_1_1/f_0029.jpg', 'tmp/tmp_1_1/f_0030.jpg', 'tmp/tmp_1_1/f_0031.jpg', 'tmp/tmp_1_1/f_0032.jpg', 'tmp/tmp_1_1/f_0033.jpg', 'tmp/tmp_1_1/f_0034.jpg', 'tmp/tmp_1_1/f_0035.jpg', 'tmp/tmp_1_1/f_0036.jpg', 'tmp/tmp_1_1/f_0037.jpg', 'tmp/tmp_1_1/f_0038.jpg', 'tmp/tmp_1_1/f_0039.jpg', 'tmp/tmp_1_1/f_0040.jpg', 'tmp/tmp_1_1/f_0041.jpg', 'tmp/tmp_1_1/f_0042.jpg', 'tmp/tmp_1_1/f_0043.jpg', 'tmp/tmp_1_1/f_0044.jpg', 'tmp/tmp_1_1/f_0045.jpg', 'tmp/tmp_1_1/f_0046.jpg', 'tmp/tmp_1_1/f_0047.jpg', 'tmp/tmp_1_1/f_0048.jpg', 'tmp/tmp_1_1/f_0049.jpg', 'tmp/tmp_1_1/f_0050.jpg', 'tmp/tmp_1_1/f_0051.jpg', 'tmp/tmp_1_1/f_0052.jpg', 'tmp/tmp_1_1/f_0053.jpg', 'tmp/tmp_1_1/f_0054.jpg', 'tmp/tmp_1_1/f_0055.jpg', 'tmp/tmp_1_1/f_0056.jpg', 'tmp/tmp_1_1/f_0057.jpg', 'tmp/tmp_1_1/f_0058.jpg', 'tmp/tmp_1_1/f_0059.jpg', 'tmp/tmp_1_1/f_0060.jpg', 'tmp/tmp_1_1/f_0061.jpg', 'tmp/tmp_1_1/f_0062.jpg', 'tmp/tmp_1_1/f_0063.jpg', 'tmp/tmp_1_1/f_0064.jpg', 'tmp/tmp_1_1/f_0065.jpg', 'tmp/tmp_1_1/f_0066.jpg', 'tmp/tmp_1_1/f_0067.jpg', 'tmp/tmp_1_1/f_0068.jpg', 'tmp/tmp_1_1/f_0069.jpg', 'tmp/tmp_1_1/f_0070.jpg', 'tmp/tmp_1_1/f_0071.jpg', 'tmp/tmp_1_1/f_0072.jpg', 'tmp/tmp_1_1/f_0073.jpg', 'tmp/tmp_1_1/f_0074.jpg', 'tmp/tmp_1_1/f_0075.jpg']

In [63]:
get_brand(sf)

0.306195
apple


'Noise'

In [22]:
avgs = []
for i in range(0, len(softmaxes)-max_frames ):
    avgs.append((sum(softmaxes[i:i+max_frames])/float(number_of_brands), frames[i:i+max_frames]))

avgs = np.array(avgs)

In [61]:
imgs

array([[[[  2.23363080e-01,  -2.36957440e-02,  -8.25192734e-02],
         [  2.23363080e-01,  -2.36957440e-02,  -8.25192734e-02],
         [  2.19441511e-01,  -2.76173126e-02,  -8.64408420e-02],
         ..., 
         [ -2.00166332e-01,  -2.58989862e-01,  -3.33499666e-01],
         [ -1.57029077e-01,  -2.15852607e-01,  -2.90362411e-01],
         [ -8.00946949e-03,  -6.68329989e-02,  -1.41342803e-01]],

        [[  2.23363080e-01,  -2.36957440e-02,  -8.25192734e-02],
         [  2.19441511e-01,  -2.76173126e-02,  -8.64408420e-02],
         [  2.19441511e-01,  -2.76173126e-02,  -8.64408420e-02],
         ..., 
         [ -1.68793783e-01,  -2.27617313e-01,  -3.02127117e-01],
         [ -3.54604499e-02,  -9.42839793e-02,  -1.68793783e-01],
         [  7.04219031e-02,   1.15983736e-02,  -6.29114303e-02]],

        [[  2.19441511e-01,  -2.76173126e-02,  -8.64408420e-02],
         [  2.19441511e-01,  -2.76173126e-02,  -8.64408420e-02],
         [  2.19441511e-01,  -2.76173126e-02,  -8.644084

In [60]:
for i in range(len(frames)):
    print frames[i], reversed_dict[np.argmax(softmaxes[i])],np.max(softmaxes[i])

tmp/tmp_1_1/f_0001.jpg nike 0.998978
tmp/tmp_1_1/f_0002.jpg nike 0.996416
tmp/tmp_1_1/f_0003.jpg cocacola 0.532543
tmp/tmp_1_1/f_0004.jpg pepsi 0.998998
tmp/tmp_1_1/f_0005.jpg apple 0.997233
tmp/tmp_1_1/f_0006.jpg apple 0.99847
tmp/tmp_1_1/f_0007.jpg apple 0.999697
tmp/tmp_1_1/f_0008.jpg apple 0.999999
tmp/tmp_1_1/f_0009.jpg apple 0.943307
tmp/tmp_1_1/f_0010.jpg apple 0.999889
tmp/tmp_1_1/f_0011.jpg apple 0.999938
tmp/tmp_1_1/f_0012.jpg apple 0.999958
tmp/tmp_1_1/f_0013.jpg apple 0.999321
tmp/tmp_1_1/f_0014.jpg apple 0.999177
tmp/tmp_1_1/f_0015.jpg apple 0.99835
tmp/tmp_1_1/f_0016.jpg apple 0.994131
tmp/tmp_1_1/f_0017.jpg apple 0.999985
tmp/tmp_1_1/f_0018.jpg apple 0.999996
tmp/tmp_1_1/f_0019.jpg apple 0.999923
tmp/tmp_1_1/f_0020.jpg pepsi 0.999993
tmp/tmp_1_1/f_0021.jpg pepsi 0.999994
tmp/tmp_1_1/f_0022.jpg apple 0.997466
tmp/tmp_1_1/f_0023.jpg apple 0.999574
tmp/tmp_1_1/f_0024.jpg apple 0.999774
tmp/tmp_1_1/f_0025.jpg apple 0.999704
tmp/tmp_1_1/f_0026.jpg apple 0.999535
tmp/tmp_1_1/f

In [27]:
top_idxs = []
new_avgs = np.array([[float(i) for i in x[0]] for x in avgs])
for i in range(number_of_brands):
    #print np.argmax(avgs[:,i])
    try:
        top_idxs.append(np.argmax(new_avgs[:,i])+number_of_brands)
        print  new_avgs[np.argmax(new_avgs[:,i])+number_of_brands]
        
    except IndexError:
        top_idxs.append(np.argmax(new_avgs[:,i]))

print top_idxs
sequences = []

[  1.48699118e-03   6.36759996e-01   3.60829026e-01   2.91857541e-05
   8.94819095e-04]
[  2.50881771e-03   2.45955110e-01   6.61510462e-03   1.44622363e-05
   7.44906545e-01]
[  1.45326578e-03   9.25543964e-01   3.37493303e-03   8.73837678e-07
   6.96269050e-02]
[  1.45326578e-03   9.25543964e-01   3.37493303e-03   8.73837678e-07
   6.96269050e-02]
[35, 63, 7, 7, 68, 63]


In [28]:
new_avgs[:,0].shape

(64,)

In [30]:
sequences = []

for top_f in top_idxs:
    sequences.append(imgs[top_f:top_f+max_frames])

sequences = np.array(sequences)
print sequences.shape
print "predicting RNN..."
ps = dp.rnn_model.predict(sequences)

(6,)
predicting RNN...


ValueError: Error when checking : expected time_distributed_1_input to have 5 dimensions, but got array with shape (6, 1)